In [17]:
import lpfet
import numpy as np
import Quant_NBody

In [18]:

x = np.arange(0.05, 2.1, 0.2)
mol1 = lpfet.Molecule(6,6, '_6ring2sites_v2')

pmv = 0.5
t, v_ext, u = lpfet.generate_from_graph(
    {0: {'v': -pmv, 'U': 1}, 1: {'v': pmv, 'U': 1}, 2: {'v': pmv, 'U': 1}, 3: {'v': -pmv, 'U': 1},
     4: {'v': pmv, 'U': 1}, 5: {'v': pmv, 'U': 1}},
    {(0, 1): 1, (1, 2): 1, (2, 3): 1, (3, 4): 1, (4, 5): 1, (0, 5): 1})
mol1.add_parameters(u, t, v_ext, [[0, 3], [1, 2, 4, 5]])


	 ====  The matrix form of a^a is built  ====


In [19]:
mol1.calculate_ks()

In [20]:
mol1.n_ks

array([0.64433757, 0.42783122, 0.42783122, 0.64433757, 0.42783122,
       0.42783122])

In [41]:
project_whole_ext_potential = True
for site_group in mol1.equiv_atom_groups.keys():
    site_id = mol1.equiv_atom_groups[site_group][0]


    y_a_correct_imp = lpfet.change_indices(mol1.y_a, site_id)
    t_correct_imp = lpfet.change_indices(mol1.t, site_id)
    v_ext_correct_imp = lpfet.change_indices(mol1.v_ext, site_id)

    P, v = Quant_NBody.Householder_transformation(y_a_correct_imp)
    if project_whole_ext_potential:
        h_tilde = P @ (t_correct_imp + np.diag(v_ext_correct_imp)) @ P
        h_tilde_dimer = h_tilde[:2, :2]
    else:
        h_tilde = P @ (t_correct_imp ) @ P
        h_tilde_dimer = h_tilde[:2, :2]
        h_tilde_dimer[0,0] += mol1.v_ext[site_id]
    u_0_dimer = np.zeros((2, 2, 2, 2), dtype=np.float64)
    # u_0_dimer[0, 0, 0, 0] += mol1.u[site_id]
    mu_imp = mol1.mu_hxc[[site_id]]  # Double parenthesis so I keep array, in future this will be list of
    # indices for block householder
    print(site_id, lpfet.cost_function_CASCI(np.array([0]), mol1.embedded_mol, h_tilde_dimer, u_0_dimer, 0)*2, mol1.n_ks[site_id])

0 0.8303418012614793 0.644337567297407
1 0.36607909936926003 0.4278312163512964


In [33]:
v_ext_correct_imp



array([ 0.5, -0.5,  0.5, -0.5,  0.5,  0.5])